In [48]:
import pandas as pd
from datetime import datetime

In [49]:
data =pd.read_csv("../data/processed/openweather_hourly/Chicago_hourly_data.csv")

In [50]:
data[data['Snow (1h)'] != 0]

,Year,Month,Day,Time,Temperature (F),Feels Like (F),Temp Min (F),Temp Max (F),Pressure,Humidity,Wind Speed,Wind Deg,Wind Gust,Clouds All,Rain (1h),Snow (1h),Weather_description
382,2024,11,20,21:00:00,33.37,28.67,31.08,34.25,1009,77,5.01,314,8.01,100,0.0,0.39,light snow
385,2024,11,21,00:00:00,32.88,21.43,30.72,34.03,1007,74,18.99,302,27.00,100,0.0,0.15,light snow
386,2024,11,21,01:00:00,31.78,20.97,29.98,33.13,1007,82,15.99,298,23.00,100,0.0,0.47,light snow
394,2024,11,21,09:00:00,30.40,17.80,28.56,31.62,1006,84,23.00,299,32.99,99,0.0,1.09,snow
395,2024,11,21,10:00:00,29.86,18.45,28.72,31.62,1006,91,16.11,260,26.46,100,0.0,1.89,snow
396,2024,11,21,11:00:00,30.88,20.62,30.02,32.76,1005,94,13.80,260,24.16,100,0.0,2.99,snow
397,2024,11,21,12:00:00,32.38,21.31,30.70,34.09,1003,93,17.27,270,23.02,100,0.0,1.54,heavy snow
398,2024,11,21,13:00:00,33.39,20.79,31.55,34.99,1002,93,28.99,279,38.99,100,0.0,0.92,snow
399,2024,11,21,14:00:00,34.36,22.06,32.52,36.12,1002,94,24.00,266,31.00,100,0.0,0.39,light snow
401,2024,11,21,16:00:00,37.09,27.41,35.29,38.62,1002,94,17.27,310,27.63,100,0.0,0.27,mist


In [51]:
restructured_simple = pd.read_csv("../data/restructured_simple/KORD.csv")
restructured_simple.columns

Index(['YEAR', 'MONTH', 'DAY', 'TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'TAVG'], dtype='object')

In [57]:
import json
import os
import pandas as pd
from datetime import datetime
import pytz
def process_weather_data(data_file, hourly_data_file):
    data = pd.read_csv(data_file)
    hourly_data = pd.read_csv(hourly_data_file)

    # Convert the 'Datetime' column in 'hourly_data' to a datetime object and extract Year, Month, Day
    hourly_data['Datetime'] = pd.to_datetime(hourly_data[['Year', 'Month', 'Day']])
    hourly_data['Year'] = hourly_data['Datetime'].dt.year
    hourly_data['Month'] = hourly_data['Datetime'].dt.month
    hourly_data['Day'] = hourly_data['Datetime'].dt.day

    start_date = pd.to_datetime('2023-11-30')
    end_date = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))

    data.loc[
        (data['YEAR'] == 2024) &
        ((data['MONTH'] == 11) & (data['DAY'] >= 12)),
        ['TAVG', 'TMIN', 'TMAX', 'PRCP']
    ] = None  # Replace values of these columns to NA for this date range

    hourly_aggregated = hourly_data.groupby(['Year', 'Month', 'Day']).agg(
        TAVG=('Temperature (F)', 'mean'),  # Average Temperature to TAVG
        TMIN=('Temp Min (F)', 'min'),      # Min Temp to TMIN
        TMAX=('Temp Max (F)', 'max'),      # Max Temp to TMAX
        PRCP=('Rain (1h)', 'sum'),        # Pressure to PRCP
        SNOW=('Snow (1h)', 'sum')
    ).reset_index()

    # Rename the columns in hourly_aggregated to avoid conflict during merge
    hourly_aggregated = hourly_aggregated.rename(columns={
        'TAVG': 'TAVG_new',
        'TMIN': 'TMIN_new',
        'TMAX': 'TMAX_new',
        'PRCP': 'PRCP_new',
        'SNOW': 'SNOW_new'
    })

    # Merge the aggregated hourly data with 'data', replacing NA values with hourly aggregated values
    data = pd.merge(data, hourly_aggregated, how='outer', left_on=['YEAR', 'MONTH', 'DAY'], right_on=['Year', 'Month', 'Day'])

    # Replace the NA values in the columns with the values from the aggregated hourly 
    data['YEAR'] = data['YEAR'].combine_first(data['Year'])
    data['MONTH'] = data['MONTH'].combine_first(data['Month'])
    data['DAY'] = data['DAY'].combine_first(data['Day'])

    data['TAVG'] = data['TAVG_new'].combine_first(data['TAVG']) 
    data['TAVG'] = data['TAVG_new'].combine_first(data['TAVG'])  # Replace NA in 'TAVG' with hourly aggregated value
    data['TMIN'] = data['TMIN_new'].combine_first(data['TMIN'])  # Replace NA in 'TMIN' with hourly aggregated value
    data['TMAX'] = data['TMAX_new'].combine_first(data['TMAX'])  # Replace NA in 'TMAX' with hourly aggregated value
    data['PRCP'] = data['PRCP_new'].combine_first(data['PRCP'])  # Replace NA in 'PRCP' with hourly aggregated value
    data['SNOW'] = data['SNOW_new'].combine_first(data['SNOW'])
    # Drop the extra columns created during the merge (e.g., columns with '_new' suffix)
    
    data = data.drop(columns=[col for col in data.columns if col.endswith('_new') or col in ['Year', 'Month', 'Day']])

    return data

In [58]:
combined = process_weather_data("../data/restructured_simple/KORD.csv","../data/processed/openweather_hourly/Chicago_hourly_data.csv")

/var/folders/p_/wj4jvr5929x__hfp1fb3gy400000gs/T/ipykernel_44717/1802361089.py:46: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  data['YEAR'] = data['YEAR'].combine_first(data['Year'])
/var/folders/p_/wj4jvr5929x__hfp1fb3gy400000gs/T/ipykernel_44717/1802361089.py:47: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  data['MONTH'] = data['MONTH'].combine_first(data['Month'])
/var/folders/p_/wj4jvr5929x__hfp1fb3gy400000gs/T/ipykernel_44717/1802361089.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version

In [59]:
combined.tail(20)

,YEAR,MONTH,DAY,TMAX,TMIN,PRCP,SNOW,SNWD,TAVG
28531,2024,11,11,59.04,39.61,0.00,0.00,0.0,50.990000
28532,2024,11,12,51.49,39.29,0.00,0.00,-9999.0,45.706667
28533,2024,11,13,53.49,36.68,6.75,0.00,-9999.0,46.242083
28534,2024,11,14,54.54,46.67,5.70,0.00,-9999.0,50.855417
28535,2024,11,15,55.49,39.61,0.00,0.00,-9999.0,48.748750
28536,2024,11,16,53.19,39.22,0.00,0.00,-9999.0,47.854167
28537,2024,11,17,61.12,43.38,1.02,0.00,-9999.0,52.195000
28538,2024,11,18,61.25,46.99,20.76,0.00,-9999.0,54.479583
28539,2024,11,19,65.25,45.37,1.18,0.00,-9999.0,58.123333
28540,2024,11,20,50.13,30.99,3.02,0.39,-9999.0,41.620000


In [46]:
import os
import pandas as pd
from datetime import datetime
import pytz
def process_and_save_data(data_json, city_name):
    # Initialize an empty list to store the rows of data
    rows = []
    timezone = pytz.timezone(city_timezones[city_name])  # Get the timezone for the city

    # Loop through each entry in the "list" field of the JSON data
    for chunk in data_json:
        for entry in chunk:
            # Extract the Unix timestamp and convert to local datetime
            dt_utc = datetime.utcfromtimestamp(entry['dt']).replace(tzinfo=pytz.utc)
            dt_local = dt_utc.astimezone(timezone)

            # Extract the components: year, month, day, time
            year = dt_local.year
            month = dt_local.month
            day = dt_local.day
            time = dt_local.strftime('%H:%M:%S')  # Time in HH:MM:SS format

            # Extract temperature, which is already in Fahrenheit
            temp = entry['main']['temp']
            feels_like = entry['main']['feels_like']
            temp_min = entry['main']['temp_min']
            temp_max = entry['main']['temp_max']

            # Extract pressure and humidity
            pressure = entry['main']['pressure']
            humidity = entry['main']['humidity']

            # Extract wind details
            wind_speed = entry['wind']['speed']
            wind_deg = entry['wind']['deg']
            wind_gust = entry['wind'].get('gust', None)

            # Extract cloud coverage
            clouds_all = entry['clouds']['all']

            # Extract rain and snow data, default to 0 if not present
            rain = entry.get('rain', {}).get('1h', 0)
            snow = entry.get('snow', {}).get('1h', 0)
            # Extract weather description
            weather_description = entry['weather'][0]['description']

            # Append the data as a row
            rows.append([
                year, month, day, time, temp, feels_like, temp_min, temp_max,
                pressure, humidity, wind_speed, wind_deg, wind_gust, clouds_all,
                rain, snow, weather_description
            ])

    # Create a DataFrame with the specified column names
    df = pd.DataFrame(rows, columns=[
        'Year', 'Month', 'Day', 'Time', 'Temperature (F)', 'Feels Like (F)',
        'Temp Min (F)', 'Temp Max (F)', 'Pressure', 'Humidity', 'Wind Speed',
        'Wind Deg', 'Wind Gust', 'Clouds All', 'Rain (1h)', 'Snow (1h)'
    ])

    # Ensure the directory exists
    directory = '../data/processed/openweather_hourly'
    os.makedirs(directory, exist_ok=True)

    # Define the file path for saving the CSV
    file_path = os.path.join(directory, f'{city_name}_hourly_data.csv')

    # Save the DataFrame to a CSV file
    df.to_csv(file_path, index=False)
    print(f"Data saved to '{file_path}'.")


In [47]:
process_and_save_data("../data/original/openweather_hourly/Chicago_hourly_data.csv", "Chicago")

NameError: name 'city_timezones' is not defined